[![gammapy](https://img.shields.io/badge/powered%20by-gammapy-orange.svg?style=flat)](https://gammapy.org/)

# Gammapy Part II: Spectral Analysis 
**Tutors:** Rubens Costa Jr.

This hands-on tutorial gives an introduction and overview of [Gammapy](https://gammapy.org/). 
In the first part of the this tutorial we learned about the basic data structures in Gammapy using [Third Fermi-LAT Catalog of High-Energy Sources (3FHL) catalog](http://fermi.gsfc.nasa.gov/ssc/data/access/lat/3FHL/), in this second part we will perform a spectral analysis using simulated CTA data.

## Preface
We recommend to follow this tutorial by **executing the code cells on your local machine**, along with the tutor. The estimated time for this part of the tutorial is ~30 minutes.

We're happy to receive any **feedback or questions** on the tutorial via mail to *rubensjrcosta@gmail.com*.


___

<a id='setup'></a>
🔝 [Back to Top](#intro)<br>
## 2. Setup 

**Important**: to run this tutorial the environment variable `GAMMAPY_DATA` must be defined and point to the directory on your machine where the datasets needed are placed. To check whether your setup is correct you can execute the following cell:



In [ ]:
import os # Miscellaneous operating system interfaces

path = os.path.expandvars("$GAMMAPY_DATA")

if not os.path.exists(path):
    raise Exception("gammapy-data repository not found!")
else:
    print("Great your setup is correct!")

In case you encounter an error, you can un-comment and execute the following cell to continue. But we recommend to set up your environment correctly as described in [getting started](https://docs.gammapy.org/dev/getting-started/index.html#download-tutorials) after you are done with this notebook.

In [ ]:
# os.environ['GAMMAPY_DATA'] = os.path.join(os.getcwd(), '..')

Now we can continue with the usual IPython notebooks and Python imports:

In [ ]:
# Display figures directly inline
%matplotlib inline 

import matplotlib.pyplot as plt # A collection of command style functions

In [ ]:
# I'd like to ignore some deprecation warnings by matplotlib
# This in general not advised, but for this specific notebook
from matplotlib import MatplotlibDeprecationWarning # A class for issuing deprecation warnings 

import warnings # Warning control
warnings.filterwarnings(
    "ignore", category=MatplotlibDeprecationWarning
)

In [ ]:
import gammapy
from astropy import units as u
import numpy as np
from astropy.io import ascii
import collections
import sys, os
import matplotlib.pyplot as plt
from gammapy.makers import SpectrumDatasetMaker, SafeMaskMaker, ReflectedRegionsBackgroundMaker
from gammapy.modeling import Fit
from gammapy.data import Observation, Observations
from gammapy.datasets import SpectrumDatasetOnOff, SpectrumDataset, Datasets
from gammapy.irf import load_cta_irfs
from gammapy.maps import MapAxis, RegionGeom, WcsNDMap, WcsGeom

from gammapy.modeling.models import (
    EBLAbsorptionNormSpectralModel,
    Models,
    PowerLawSpectralModel,
    SkyModel,
    ExpCutoffPowerLawSpectralModel
)

from gammapy.irf import EffectiveAreaTable2D

from numpy.random import RandomState

from scipy.stats import chi2, norm

from gammapy.estimators import FluxPointsEstimator
from gammapy.estimators import FluxPoints
from gammapy.datasets import FluxPointsDataset

# astropy imports
from astropy.coordinates import SkyCoord, Angle
from astropy import units as u
from astropy.io import fits
from astropy.table import Table, Column

from gammapy.estimators import SensitivityEstimator

# astropy affiliated packages imports
from regions import CircleSkyRegion

from gammapy.stats import WStatCountsStatistic
from gammapy.stats import CashCountsStatistic
from scipy.stats import sem
from gammapy.maps import Map

import math

import pandas as pd

In [ ]:
# Define simulation parameters parameters
livetimes = [2.21] * u.h # Livetime exposure of the simulated observation
sites = ['North'] # CTA North
irftime = '0.5h' #  Observation times

In [ ]:
# Analysis parameters
on_region_radius = Angle("0.11 deg")
alpha = 0.2  # acceptance_off=1./alpha) # Relative background efficiency in the off region
emin = 1 * u.GeV
emax = 100. * u.TeV

In [ ]:
# We simulate each observation n_obs times, to randomize the renortets:
n_obs = 1

In [ ]:
df_simulated = pd.read_csv('bllac_norte_1x/bllac_norte_1x.txt', sep='  ', engine='python')
display(df_simulated)

In [ ]:
selected_obs = df_simulated.sample(n = 2).reset_index(drop=True)
selected_obs

In [ ]:
selected_obs.to_string('bllac_norte_1x/selected_obs.txt', index=False)

In [ ]:
source_name, ra_pointing, dec_pointing, ra, dec, red, luminosity, index, amplitude, IRF_zen, offset = np.loadtxt('bllac_norte_1x/selected_obs.txt', skiprows=1, unpack=True)

In [ ]:
VERBOSE = True

In [ ]:
fitbackend = 'minuit'
optimize_opts = {'tol': 0.1, 'strategy': 1, 'print_level': 3}

In [ ]:
output = {'Source_Name': [],
          'RA_pointing': [],
          'DEC_pointing': [],
          'RAJ2000': [],
          'DEJ2000': [],
          'Redshift': [],
          'Amplitude': [],
          'Index': [],
          'Luminosity': [],
          'Offset': [],
          'Site': [],
          'Irf_Zen_pointing': [],
          'Irf_Time': [],
          'Livetime_pointing': [],
          'Emin': [],
          'Ethreshold': [],
          'AboveEthFlag': [],
          'Model_Name': [],
          'N_sims' : [],
          'N_failed' : [],
          'Bkg': [],
          'Significance': [],
          'Energy0': [],
          'Energy1': [],
          'eref0': [],
          'eref1': [],
          'eref2': [],
          'eref3': [],
          'eref4': [],
          'eref5': [],
          'eref6': [],
          'eref7': [],
          'eref8': [],
          'eref9': [],
          'eref10': [],
          'eref11': [],
          'emin0': [],
          'emin1': [],
          'emin2': [],
          'emin3': [],
          'emin4': [],
          'emin5': [],
          'emin6': [],
          'emin7': [],
          'emin8': [],
          'emin9': [],
          'emin10': [],
          'emin11': [],
          'emax0': [],
          'emax1': [],
          'emax2': [],
          'emax3': [],
          'emax4': [],
          'emax5': [],
          'emax6': [],
          'emax7': [],
          'emax8': [],
          'emax9': [],
          'emax10': [],
          'emax11': [],
          'dnde0': [],
          'dnde1': [],
          'dnde2': [],
          'dnde3': [],
          'dnde4': [],
          'dnde5': [],
          'dnde6': [],
          'dnde7': [],
          'dnde8': [],
          'dnde9': [],
          'dnde10': [],
          'dnde11': [],
          'dnde_err0': [],
          'dnde_err1': [],
          'dnde_err2': [],
          'dnde_err3': [],
          'dnde_err4': [],
          'dnde_err5': [],
          'dnde_err6': [],
          'dnde_err7': [],
          'dnde_err8': [],
          'dnde_err9': [],
          'dnde_err10': [],
          'dnde_err11': [],
          'amplitude_fit': [],
          'amplitude_err_fit': [],
          'index_fit': [],
          'index_err_fit': [],
          'reference_fit': [],
          'int_sens': []

      }

In [ ]:
#for i in range(len(name)):
for i in range(len(selected_obs)):
    print(f"""#################### source_name = {source_name[i]} ####################""")
    
    # Define Target Region
    # source position
    src_position = SkyCoord(
        ra[i], 
        dec[i], 
        unit="deg", 
        frame="icrs"
    )
    
    # observation region
    on_region = CircleSkyRegion(
        center=src_position, 
        radius=on_region_radius
    )

    
    # Define simulation parameters
    pointing = SkyCoord(
         ra_pointing[i], 
         dec_pointing[i], 
         unit="deg", 
         frame="icrs"
     )
    
    # Define spectral model - a simple Power Law in this case
    specmodel = PowerLawSpectralModel(
        amplitude=amplitude[i] * u.Unit('cm^-2 s^-1 MeV^-1'),
        reference=1 * u.Unit('MeV'),
        index=index[i]
    )
    
    # EBL absorbption spectral model
    absorption = EBLAbsorptionNormSpectralModel.read_builtin(
        "finke", # Available model in gammapy-data:{'franceschini', 'dominguez', 'finke'}
        redshift=red[i]
    )
    
    absspecmodel = specmodel * absorption

    skymodel = SkyModel(
        spectral_model=absspecmodel, 
        name="crab"
    )
    
    obs1 = Observation.create(
        pointing=pointing, 
        livetime=livetimes, # Livetime exposure of the simulated observation
        irfs=load_cta_irfs('/home/bornagain/Documents/GitHub/gammapy/gammapy-notebooks/0.20.1/tutorials/data/caldb/data/cta/prod3b-v2/bcf/North_z'+str(int(IRF_zen[i]))+'_0.5h/irf_file.fits')
    )
    obs = [obs1]

    AboveEthFlag = True
    
    # reconstructed energy
    energy_reco = MapAxis.from_energy_bounds(
        emin, 
        emax, 
        nbin=5, 
        per_decade=True, 
        name="energy"
    )
    
    # true energy should be wider than reconstructed energy
    energy_true = MapAxis.from_energy_bounds(
        0.3*emin, 
        3*emax, 
        nbin=8, 
        per_decade=True, 
        name="energy_true"
    )

    significances = []

    # Make the SpectrumDataset from observation parameters
    geom = RegionGeom.create(
        region=on_region, 
        axes=[energy_reco]
    )
    
    # Make the SpectrumDataset
    dataset_empty = SpectrumDataset.create(
        geom=geom, 
        energy_axis_true=energy_true, 
        name=int(source_name[i])
    )
    
    # The irfs and background are computed at a single fixed offset
    maker = SpectrumDatasetMaker(
        # Apply containment correction for point sources and circular on regions
        containment_correction=True, 
        # Selecting which maps to make
        selection=["edisp", "background", "exposure"]
    )
    
    # Make safe data range mask for a given observation
    safe_maker = SafeMaskMaker(methods=["bkg-peak"])

    n_sims = 0
    n_failed = 0
    significance=[]
    eref0 = []
    eref1 = []
    eref2 = []
    eref3 = []
    eref4 = []
    eref5 = []
    eref6 = []
    eref7 = []
    eref8 = []
    eref9 = []
    eref10 = []
    eref11 = []
    emin0 = []
    emin1 = []
    emin2 = []
    emin3 = []
    emin4 = []
    emin5 = []
    emin6 = []
    emin7 = []
    emin8 = []
    emin9 = []
    emin10 = []
    emin11 = []
    emax0 = []
    emax1 = []
    emax2 = []
    emax3 = []
    emax4 = []
    emax5 = []
    emax6 = []
    emax7 = []
    emax8 = []
    emax9 = []
    emax10 = []
    emax11 = []
    dnde0 = []
    dnde1 = []
    dnde2 = []
    dnde3 = []
    dnde4 = []
    dnde5 = []
    dnde6 = []
    dnde7 = []
    dnde8 = []
    dnde9 = []
    dnde10 = []
    dnde11 = []
    dnde_err0 = []
    dnde_err1 = []
    dnde_err2 = []
    dnde_err3 = []
    dnde_err4 = []
    dnde_err5 = []
    dnde_err6 = []
    dnde_err7 = []
    dnde_err8 = []
    dnde_err9 = []
    dnde_err10 = []
    dnde_err11 = []
    index_fit= []
    amplitude_fit = []
    amplitude_err_fit = []
    index_fit = []
    index_err_fit = []
    reference_fit = []
    datasets = Datasets()
    
    while n_sims < n_obs:
        if VERBOSE:
            print(f"""#################### n_sims = {n_sims+1} ####################""")
            for obs_id in obs:
                
                dataset = maker.run(
                    dataset_empty, 
                    obs_id
                )
                
                dataset = safe_maker.run(
                    dataset, 
                    obs_id
                )
                
                # Energy range maps defined by the mask_safe only
                low_energies_safe = dataset.energy_range_safe[0]
                
                # Energy threshold
                energy_threshold = low_energies_safe.get_by_coord({'skycoord': src_position})[0] * low_energies_safe.unit
                
                # Create spectrum dataseton off from another dataset
                dataset_onoff = SpectrumDatasetOnOff.from_spectrum_dataset(
                    dataset=dataset, 
                    acceptance=1, # Relative background efficiency in the on region
                    acceptance_off=1./alpha
                ) # Relative background efficiency in the off region (alpha = 0.2)
                
                # Add the model on the dataset
                skymodel_fit = skymodel.copy()
                dataset_onoff.models = skymodel_fit

                # Simulate fake counts (on and off) for the current model and reduced IRFs
                try:
                    dataset_onoff.fake(
                        random_state='random-seed', # Defines random number generator initialisation.
                        npred_background=dataset.npred_background() # Predicted background counts
                    )
                except ValueError as e:
                    print(f'DEBUG JPL: In fake: Caught {e}')
                    n_failed += 1
                    continue
                
                # append dataset_onoff to the datasets
                datasets.append(dataset_onoff)
            
            #  stack all dataset
            dataset_stacked = Datasets(datasets).stack_reduce(name=int(source_name[i]))
            
            # copy skymodel to fit it
            skymodel_fit = skymodel.copy()
            
            # loads the model into the staked dataset
            dataset_stacked.models = skymodel_fit

            skymodel_fit.parameters["reference"].quantity = 0.2 * u.TeV
            skymodel_fit.parameters["amplitude"].value *= (1.+(2.*np.random.rand(1)-1.)/10.)
            skymodel_fit.parameters["index"].value *= (1.+(2.*np.random.rand(1)-1.)/10.)

            stacked_fit = Fit(backend=fitbackend, optimize_opts=optimize_opts)
            renortet_stacked = stacked_fit.run([dataset_stacked])
            print(renortet_stacked.parameters["amplitude"].quantity.value)

            amplitude_fit.append(renortet_stacked.parameters["amplitude"].quantity.value)
            amplitude_err_fit.append(renortet_stacked.parameters["amplitude"].error)
            index_fit.append(renortet_stacked.parameters["index"].quantity.value)
            index_err_fit.append(renortet_stacked.parameters["index"].error)
            reference_fit.append(renortet_stacked.parameters["reference"].quantity.value)

            spectral_model = PowerLawSpectralModel(
                amplitude=amplitude[i] * u.Unit('cm^-2 s^-1 MeV^-1'),
                reference=1 * u.Unit('MeV'),
                index=index[i]
            )

            absorption_ebl = EBLAbsorptionNormSpectralModel.read_builtin("finke", redshift=red[i])
            absspecmodel_ebl = spectral_model * absorption_ebl
            
            model = SkyModel(spectral_model=absspecmodel_ebl, name="crab")

            dataset_stacked.models = [model]
            
            
            # Estimates flux points for a given list of datasets, energies and spectral model
            energy_edges = MapAxis.from_energy_bounds("0.1 TeV", "30 TeV", nbin=12).edges
            fpe = FluxPointsEstimator(energy_edges=energy_edges, source="crab", selection_optional="all")
            flux_points = fpe.run(datasets=dataset_stacked)
            print(flux_points.to_table(sed_type="dnde", formatted=True))
            print(flux_points["dnde"].data)
            
            # Saves the flux points
            file = open("/home/bornagain/Documents/GitHub/gammapy/gammapy-notebooks/0.20.1/presentations/gammapy/results/flux_point_src"+str(int(source_name[i]))+".txt","+w")
            file.write('E_ref (TeV) E_max (TeV) E_min (TeV) dnde (cm-2 s-1 MeV-1) dnde_err \r\n')

            for j in range(len(flux_points["energy_max"].value)):
                if flux_points["dnde"].data[j] > 0:
                    file.write(str(flux_points["energy_ref"].value[j]) + '  ' + str(flux_points["energy_max"].value[j]) + '  ' + str(flux_points["energy_min"].value[j]) + '  ' + str(flux_points["dnde"].data[j])[1:-1] + '  ' + str(flux_points["dnde_err"].data[j])[1:-1] + '\n')

            eref0.append(flux_points["energy_ref"].data[0])
            eref1.append(flux_points["energy_ref"].data[1])
            eref2.append(flux_points["energy_ref"].data[2])
            eref3.append(flux_points["energy_ref"].data[3])
            eref4.append(flux_points["energy_ref"].data[4])
            eref5.append(flux_points["energy_ref"].data[5])
            eref6.append(flux_points["energy_ref"].data[6])
            eref7.append(flux_points["energy_ref"].data[7])
            eref8.append(flux_points["energy_ref"].data[8])
            eref9.append(flux_points["energy_ref"].data[9])
            eref10.append(flux_points["energy_ref"].data[10])
            eref11.append(flux_points["energy_ref"].data[11])
            emin0.append(flux_points["energy_min"].data[0])
            emin1.append(flux_points["energy_min"].data[1])
            emin2.append(flux_points["energy_min"].data[2])
            emin3.append(flux_points["energy_min"].data[3])
            emin4.append(flux_points["energy_min"].data[4])
            emin5.append(flux_points["energy_min"].data[5])
            emin6.append(flux_points["energy_min"].data[6])
            emin7.append(flux_points["energy_min"].data[7])
            emin8.append(flux_points["energy_min"].data[8])
            emin9.append(flux_points["energy_min"].data[9])
            emin10.append(flux_points["energy_min"].data[10])
            emin11.append(flux_points["energy_min"].data[11])
            emax0.append(flux_points["energy_max"].data[0])
            emax1.append(flux_points["energy_max"].data[1])
            emax2.append(flux_points["energy_max"].data[2])
            emax3.append(flux_points["energy_max"].data[3])
            emax4.append(flux_points["energy_max"].data[4])
            emax5.append(flux_points["energy_max"].data[5])
            emax6.append(flux_points["energy_max"].data[6])
            emax7.append(flux_points["energy_max"].data[7])
            emax8.append(flux_points["energy_max"].data[8])
            emax9.append(flux_points["energy_max"].data[9])
            emax10.append(flux_points["energy_max"].data[10])
            emax11.append(flux_points["energy_max"].data[11])
            dnde0.append(flux_points["dnde"].data[0])
            dnde1.append(flux_points["dnde"].data[1])
            dnde2.append(flux_points["dnde"].data[2])
            dnde3.append(flux_points["dnde"].data[3])
            dnde4.append(flux_points["dnde"].data[4])
            dnde5.append(flux_points["dnde"].data[5])
            dnde6.append(flux_points["dnde"].data[6])
            dnde7.append(flux_points["dnde"].data[7])
            dnde8.append(flux_points["dnde"].data[8])
            dnde9.append(flux_points["dnde"].data[9])
            dnde10.append(flux_points["dnde"].data[10])
            dnde11.append(flux_points["dnde"].data[11])
            dnde_err0.append(flux_points["dnde_err"].data[0])
            dnde_err1.append(flux_points["dnde_err"].data[1])
            dnde_err2.append(flux_points["dnde_err"].data[2])
            dnde_err3.append(flux_points["dnde_err"].data[3])
            dnde_err4.append(flux_points["dnde_err"].data[4])
            dnde_err5.append(flux_points["dnde_err"].data[5])
            dnde_err6.append(flux_points["dnde_err"].data[6])
            dnde_err7.append(flux_points["dnde_err"].data[7])
            dnde_err8.append(flux_points["dnde_err"].data[8])
            dnde_err9.append(flux_points["dnde_err"].data[9])
            dnde_err10.append(flux_points["dnde_err"].data[10])
            dnde_err11.append(flux_points["dnde_err"].data[11])
            
            # Extract analysis statistics
            significancee = WStatCountsStatistic(n_on=sum(dataset_stacked.counts.data), n_off=sum(dataset_stacked.counts_off.data), alpha=0.2).sqrt_ts
            significance.append(significancee)
 
            n_sims += 1
            print(np.median(significance))

    output['Source_Name'].append(int(source_name[i]))
    output['RA_pointing'].append(pointing.ra.value)
    output['DEC_pointing'].append(pointing.dec.value)
    output['RAJ2000'].append(src_position.ra.value)
    output['DEJ2000'].append(src_position.dec.value)
    output['Redshift'].append(absorption.redshift.value)
    output['Amplitude'].append(specmodel.amplitude.value)
    output['Luminosity'].append(luminosity[i])
    output['Offset'].append(offset[i])
    output['Index'].append(specmodel.index.value)
    output['Site'].append(sites)
    output['Irf_Zen_pointing'].append(int(IRF_zen[i]))
    output['Irf_Time'].append(float(irftime.split('h')[0]))
    output['Ethreshold'].append(energy_threshold.value)
    output['AboveEthFlag'].append(AboveEthFlag)
    output['N_sims'].append(n_sims)
    output['N_failed'].append(n_failed)
    output['Significance'].append(np.median(significance))
    output['eref0'].append(eref0)
    output['eref1'].append(eref1)
    output['eref2'].append(eref2)
    output['eref3'].append(eref3)
    output['eref4'].append(eref4)
    output['eref5'].append(eref5)
    output['eref6'].append(eref6)
    output['eref7'].append(eref7)
    output['eref8'].append(eref8)
    output['eref9'].append(eref9)
    output['eref10'].append(eref10)
    output['eref11'].append(eref11)
    output['emin0'].append(emin0)
    output['emin1'].append(emin1)
    output['emin2'].append(emin2)
    output['emin3'].append(emin3)
    output['emin4'].append(emin4)
    output['emin5'].append(emin5)
    output['emin6'].append(emin6)
    output['emin7'].append(emin7)
    output['emin8'].append(emin8)
    output['emin9'].append(emin9)
    output['emin10'].append(emin10)
    output['emin11'].append(emin11)
    output['emax0'].append(emax0)
    output['emax1'].append(emax1)
    output['emax2'].append(emax2)
    output['emax3'].append(emax3)
    output['emax4'].append(emax4)
    output['emax5'].append(emax5)
    output['emax6'].append(emax6)
    output['emax7'].append(emax7)
    output['emax8'].append(emax8)
    output['emax9'].append(emax9)
    output['emax10'].append(emax10)
    output['emax11'].append(emax11)
    output['dnde0'].append(dnde0)
    output['dnde1'].append(dnde1)
    output['dnde2'].append(dnde2)
    output['dnde3'].append(dnde3)
    output['dnde4'].append(dnde4)
    output['dnde5'].append(dnde5)
    output['dnde6'].append(dnde6)
    output['dnde7'].append(dnde7)
    output['dnde8'].append(dnde8)
    output['dnde9'].append(dnde9)
    output['dnde10'].append(dnde10)
    output['dnde11'].append(dnde11)
    output['dnde_err0'].append(dnde_err0)
    output['dnde_err1'].append(dnde_err1)
    output['dnde_err2'].append(dnde_err2)
    output['dnde_err3'].append(dnde_err3)
    output['dnde_err4'].append(dnde_err4)
    output['dnde_err5'].append(dnde_err5)
    output['dnde_err6'].append(dnde_err6)
    output['dnde_err7'].append(dnde_err7)
    output['dnde_err8'].append(dnde_err8)
    output['dnde_err9'].append(dnde_err9)
    output['dnde_err10'].append(dnde_err10)
    output['dnde_err11'].append(dnde_err11)
    output['amplitude_fit'].append(amplitude_fit)
    output['amplitude_err_fit'].append(amplitude_err_fit)
    output['index_fit'].append(index_fit)
    output['index_err_fit'].append(index_err_fit)
    output['reference_fit'].append(reference_fit)


In [ ]:
# Save renortets
renortets = Table()
renortets['Source_Name'] = Column(output['Source_Name'], description='Source name')
renortets['RA_pointing'] = Column(output['RA_pointing'], format='{:.6f}', unit='degree', description='Right Ascension')
renortets['DEC_pointing'] = Column(output['DEC_pointing'], format='{:.6f}', unit='degree', description='Declination')
renortets['RAJ2000'] = Column(output['RAJ2000'], format='{:.6f}', unit='degree', description='Right Ascension')
renortets['DEJ2000'] = Column(output['DEJ2000'], format='{:.6f}', unit='degree', description='Declination')
renortets['Redshift'] = Column(output['Redshift'], format='{:.4f}', unit=None, description='Redshift')
renortets['Amplitude'] = Column(output['Amplitude'], format='{:.4f}', unit=None, description='Amplitude')
renortets['Index'] = Column(output['Index'], format='{:.4f}', unit=None, description='Index')
renortets['Luminosity'] = Column(output['Luminosity'], format='{:.4f}', unit=None, description='Luminosity')
renortets['Offset'] = Column(output['Offset'], format='{:.6f}', unit=None, description='Offset')
renortets['Site'] = Column(output['Site'], description='Site of the considered CTA array')
renortets['Ethreshold'] = Column(output['Ethreshold'], unit='TeV', description='Effective energy threshold')
renortets['EthFlag'] = Column(output['AboveEthFlag'], unit=None, description='Is requested minimal energy high enough, i.e. above energy threshold from IRF ?')
renortets['N_sims'] = Column(output['N_sims'], unit=None, description='Number of successfully thrown simulations')
renortets['N_failed'] = Column(output['N_failed'], unit=None, description='Number of failed simulations')
renortets['Irf_Zen_pointing'] = Column(output['Irf_Zen_pointing'], unit='deg', description='Irf_Zen_pointing')
renortets['Significance'] = Column(output['Significance'], format='{:.2f}', unit='', description='Median significance')
renortets['eref0'] = Column(output['eref0'], format='{:.2f}', unit='', description='eref')
renortets['eref1'] = Column(output['eref1'], format='{:.2f}', unit='', description='eref')
renortets['eref2'] = Column(output['eref2'], format='{:.2f}', unit='', description='eref')
renortets['eref3'] = Column(output['eref3'], format='{:.2f}', unit='', description='eref')
renortets['eref4'] = Column(output['eref4'], format='{:.2f}', unit='', description='eref')
renortets['eref5'] = Column(output['eref5'], format='{:.2f}', unit='', description='eref')
renortets['eref6'] = Column(output['eref6'], format='{:.2f}', unit='', description='eref')
renortets['eref7'] = Column(output['eref7'], format='{:.2f}', unit='', description='eref')
renortets['eref8'] = Column(output['eref8'], format='{:.2f}', unit='', description='eref')
renortets['eref9'] = Column(output['eref9'], format='{:.2f}', unit='', description='eref')
renortets['eref10'] = Column(output['eref10'], format='{:.2f}', unit='', description='eref')
renortets['eref11'] = Column(output['eref11'], format='{:.2f}', unit='', description='eref')
renortets['emax0'] = Column(output['emax0'], format='{:.2f}', unit='', description='emax')
renortets['emax1'] = Column(output['emax1'], format='{:.2f}', unit='', description='emax')
renortets['emax2'] = Column(output['emax2'], format='{:.2f}', unit='', description='emax')
renortets['emax3'] = Column(output['emax3'], format='{:.2f}', unit='', description='emax')
renortets['emax4'] = Column(output['emax4'], format='{:.2f}', unit='', description='emax')
renortets['emax5'] = Column(output['emax5'], format='{:.2f}', unit='', description='emax')
renortets['emax6'] = Column(output['emax6'], format='{:.2f}', unit='', description='emax')
renortets['emax7'] = Column(output['emax7'], format='{:.2f}', unit='', description='emax')
renortets['emax8'] = Column(output['emax8'], format='{:.2f}', unit='', description='emax')
renortets['emax9'] = Column(output['emax9'], format='{:.2f}', unit='', description='emax')
renortets['emax10'] = Column(output['emax10'], format='{:.2f}', unit='', description='emax')
renortets['emax11'] = Column(output['emax11'], format='{:.2f}', unit='', description='emax')
renortets['emin0'] = Column(output['emin0'], format='{:.2f}', unit='', description='emin')
renortets['emin1'] = Column(output['emin1'], format='{:.2f}', unit='', description='emin')
renortets['emin2'] = Column(output['emin2'], format='{:.2f}', unit='', description='emin')
renortets['emin3'] = Column(output['emin3'], format='{:.2f}', unit='', description='emin')
renortets['emin4'] = Column(output['emin4'], format='{:.2f}', unit='', description='emin')
renortets['emin5'] = Column(output['emin5'], format='{:.2f}', unit='', description='emin')
renortets['emin6'] = Column(output['emin6'], format='{:.2f}', unit='', description='emin')
renortets['emin7'] = Column(output['emin7'], format='{:.2f}', unit='', description='emin')
renortets['emin8'] = Column(output['emin8'], format='{:.2f}', unit='', description='emin')
renortets['emin9'] = Column(output['emin9'], format='{:.2f}', unit='', description='emin')
renortets['emin10'] = Column(output['emin10'], format='{:.2f}', unit='', description='emin')
renortets['emin11'] = Column(output['emin11'], format='{:.2f}', unit='', description='emin')
renortets['dnde0'] = Column(output['dnde0'], format='{:.2f}', unit='', description='dnde')
renortets['dnde1'] = Column(output['dnde1'], format='{:.2f}', unit='', description='dnde')
renortets['dnde2'] = Column(output['dnde2'], format='{:.2f}', unit='', description='dnde')
renortets['dnde3'] = Column(output['dnde3'], format='{:.2f}', unit='', description='dnde')
renortets['dnde4'] = Column(output['dnde4'], format='{:.2f}', unit='', description='dnde')
renortets['dnde5'] = Column(output['dnde5'], format='{:.2f}', unit='', description='dnde')
renortets['dnde6'] = Column(output['dnde6'], format='{:.2f}', unit='', description='dnde')
renortets['dnde7'] = Column(output['dnde7'], format='{:.2f}', unit='', description='dnde')
renortets['dnde8'] = Column(output['dnde8'], format='{:.2f}', unit='', description='dnde')
renortets['dnde9'] = Column(output['dnde9'], format='{:.2f}', unit='', description='dnde')
renortets['dnde10'] = Column(output['dnde10'], format='{:.2f}', unit='', description='dnde')
renortets['dnde11'] = Column(output['dnde11'], format='{:.2f}', unit='', description='dnde')
renortets['dnde_err0'] = Column(output['dnde_err0'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err1'] = Column(output['dnde_err1'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err2'] = Column(output['dnde_err2'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err3'] = Column(output['dnde_err3'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err4'] = Column(output['dnde_err4'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err5'] = Column(output['dnde_err5'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err6'] = Column(output['dnde_err6'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err7'] = Column(output['dnde_err7'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err8'] = Column(output['dnde_err8'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err9'] = Column(output['dnde_err9'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err10'] = Column(output['dnde_err10'], format='{:.2f}', unit='', description='dnde_err')
renortets['dnde_err11'] = Column(output['dnde_err11'], format='{:.2f}', unit='', description='dnde_err')
renortets['amplitude_fit'] = Column(output['amplitude_fit'], format='{:.2f}', unit='', description='amplitude_fit')
renortets['amplitude_err_fit'] = Column(output['amplitude_err_fit'], format='{:.2f}', unit='', description='amplitude_err_fit')
renortets['index_fit'] = Column(output['index_fit'], format='{:.2f}', unit='', description='index_fit')
renortets['index_err_fit'] = Column(output['index_err_fit'], format='{:.2f}', unit='', description='index_err_fit')
renortets['reference_fit'] = Column(output['reference_fit'], format='{:.2f}', unit='', description='reference')

In [ ]:
renortets

In [ ]:
file = open("/home/bornagain/Documents/GitHub/gammapy/gammapy-notebooks/0.20.1/presentations/gammapy/results/North_PL_D11_1x.txt","+w")
file.write('Source_Name  RA  DEC  Redshift  Luminosity  Index  Amplitude  E_th  Significance  \r\n')

for i in range(len(renortets['RAJ2000'])):
    file.write(str(int(source_name[i])) + '  ' + str(renortets['RAJ2000'][i]) + '  ' + str(renortets['DEJ2000'][i]) + '  ' + str(renortets['Redshift'][i]) + '  ' + str(renortets['Luminosity'][i]) + '  ' + str(renortets['Index'][i]) + '  ' + str(renortets['Amplitude'][i]) + '  ' + str(renortets['Ethreshold'][i]) + '  ' + str(renortets['Significance'][i]) + '\n')

In [ ]:
df_result = pd.read_csv('results/North_PL_D11_1x.txt', sep='  ', engine='python')
display(df_result)

In [ ]:
display(selected_obs)

___

🔝 [Back to Top](#intro)<br>